# task 2.1 Model
Implement the model in C++ (allowed library for matrix computation: Eigen) or Python (allowed library for matrix computation: numpy).

In [70]:
import numpy as np
states = ['rainy', 'not_rainy'] #0= rainy 1=not_rainy
observations = ('umbrella', 'no_umbrella') # 0=umbrella. 1 =no_umbrella
pi =np.array( [0.5,0.5])
A = np.array([[0.7, 0.3],[0.3, 0.7]]) #Transmission probability 
#B = np.array([[0.9, 0.2],[0.1, 0.8]]) #Emission probability
O_true= np.array([[0.9, 0],[0, 0.2]])

# task 2.2 Direct Sampling 
Implement in the chosen programming language direct sampling, as studied in class, and use it to sample at least 15 sequences of length 20.
Note: store both the sequence of states and observations.

First, we generate random values for both R and U.


if it is a first day, 0.5 is decision range, which means that if random value is less than 0,5, True is assigned; otherwise - False.


For other days, we use P(R_t=true| R_(t-1)) as a decision range for assingning R_t, and P(U_t=true| R_t) for U_t

In [102]:
def sampling(pi,Trans,Emis,num_seq,num_t):
    R=np.zeros((num_seq, num_t),dtype='i') #R stores values for Rain
    U=np.zeros((num_seq, num_t),dtype='i') 
    for sequence in range(num_seq):
        rand_for_R=np.random.randint(100, size=num_t)/100 #randomly generate array with size num_t(20)
        rand_for_U=np.random.randint(100, size=num_t)/100 # each element is less than 1 with 2 decimal places
        
        for t in range(num_t):
            if t==0: #if it's 1st day, probabilities are [0.5 0.5]. 
                true_range_R= pi[0]
            else:#otherwise, we look at the probabilities of the previous day(t-1)
                if R[sequence][t-1]==0:#R_t-1=rainy
                    true_range_R=Trans[0][0] #P(R_t|R_(t-1)
                else:#R_t-1=not_rainy
                    true_range_R=Trans[0][1]
            if rand_for_R[t] < true_range_R:    
                R[sequence][t]=0 #R_t=rainy
            else:
                R[sequence][t]=1 #R_t=not_rainy
                #The same for observation
            if R[sequence][t]==0: #if R_t=rainy, use P(U_t=true|R_t=true)<=>O_true[0][0]
                true_range_U=Emis[0][0]
            else:#if R_t=not_rainy, use P(U_t=true|R_t=false)<=>O_true[1][1]
                true_range_U=Emis[1][1]
            if rand_for_U[t] < true_range_U:              
                U[sequence][t]=0 #U_t=umbrella
            else:
                U[sequence][t]=1
    return R, U      

In [103]:
num_t=20 #lenght of samples, 20 for this assignment 
num_seq=15 #number of sequences, min 15 for this assignment 
R,U=sampling(pi,A,O_true,num_seq,num_t)
for i in range(num_t):
    print(" ", states[R[0][i]] ," => ",observations[U[0][i]])
#print("Rain? :",list(map(lambda y: states[y], R[0][:] ))   )
#print("Umbrella? :", list(map(lambda s: observations[s], U[0][:])))

  rainy  =>  umbrella
  rainy  =>  umbrella
  not_rainy  =>  no_umbrella
  not_rainy  =>  umbrella
  not_rainy  =>  no_umbrella
  not_rainy  =>  umbrella
  not_rainy  =>  umbrella
  not_rainy  =>  umbrella
  rainy  =>  umbrella
  rainy  =>  umbrella
  not_rainy  =>  no_umbrella
  rainy  =>  umbrella
  not_rainy  =>  no_umbrella
  rainy  =>  umbrella
  not_rainy  =>  umbrella
  rainy  =>  umbrella
  not_rainy  =>  no_umbrella
  rainy  =>  umbrella
  not_rainy  =>  no_umbrella
  rainy  =>  no_umbrella


# task 2.3 Basic Forward-Backward Algorithm
Implement in the chosen programming language the basic version of the
forward-backward algorithm, as studied in class, and test it over one or more sequences that you generated.
Note: verify how good your estimate of the state is using the state sequence stored in point 2.

In [104]:
def normalize(prob):
    return [prob[i]/sum(prob) for i in range(len(prob))]   

def forward(Observers, T, O_true, pi):
    f=np.zeros((len(Observers)+1, 2 ),dtype='f')
    f[0][:]=pi
    O_false=np.eye(2, dtype=int)-O_true # P(U_t=false|R_t)
    for t in range(len(Observers)):
        if Observers[t]==1:
            O=O_false
        else:
            O=O_true
        f[t+1]=normalize(np.matmul(np.matmul(O,T),f[t])) #f_(t+1)=<(O*T)*f_t>
    return f

def backward(Observers, T, O_true, pi):
    b=np.zeros((len(Observers)+1, 2 ),dtype='f')
    b[len(Observers)][:]=[1,1]
    O_false=np.eye(2, dtype=int)-O_true
    for t in range(len(Observers),0,-1):
        if Observers[t-1]==1:
            O=O_false
        else:
            O=O_true
        b[t-1]=normalize(np.matmul(np.matmul(T,O),b[t]))
    return b

In [136]:
def forward_backward(Observers, T, O_true, pi):
    alpha = forward(Observers, T, O_true, pi)
    beta  = backward(Observers, T, O_true, pi) 
    final=np.zeros((len(Observers)+1, 2 ),dtype='f')
   
    for t in range(len(Observers)+1) :
        final[t] =normalize(np.multiply(alpha[t],beta[t]))
        #print("alpha ",alpha[t]," beta ", beta[t]," multip ", np.argmax(final[t]))
    return final

In [137]:
Obs=[0,0,1,0,0]
#print(forward_backward(U[0], A, O_true, pi))
print(forward_backward(Obs, A, O_true, pi))

[[0.6469356  0.35306445]
 [0.8673389  0.13266112]
 [0.8204191  0.17958096]
 [0.30748355 0.69251645]
 [0.8204191  0.17958093]
 [0.8673389  0.1326611 ]]


 [bonus task]Improvementing space complexity by propagating backward the forward message

In [138]:
def forward1(Observers, T, O_true, pi):
    f=pi
    O_false=np.eye(2, dtype=int)-O_true # P(U_t=false|R_t)
    for t in range(len(Observers)):
        if Observers[t]==1:
            O=O_false
        else:
            O=O_true
        f=normalize(np.matmul(np.matmul(O,T),f)) #f_(t+1)=<(O*T)*f_t>
    return f

def backward1(f1, Observers, T, O_true, pi):
    b=np.zeros((len(Observers)+1, 2 ),dtype='f')
    f=np.zeros((len(Observers)+1, 2 ),dtype='f')
    b[len(Observers)][:]=[1,1] 
    f[len(Observers)][:]=f1
    O_false=np.eye(2, dtype=int)-O_true
    for t in range(len(Observers),0,-1):
        if Observers[t-1]==1:
            O=O_false
        else:
            O=O_true
        O_inv=np.linalg.inv(O)
        T_inv=np.linalg.inv(T)
        f[t-1]=normalize(np.matmul(np.matmul(T_inv,O_inv),f[t]))
        b[t-1]=normalize(np.matmul(np.matmul(T,O),b[t]))
        
    return f,b

In [139]:
def forward_backward_impr1(Observers, T, O_true, pi):
    f = forward1(Observers, T, O_true, pi)
    alpha,beta  = backward1(f,Observers, T, O_true, pi)
    final=np.zeros((len(Observers)+1, 2 ),dtype='f')
    for t in range(len(Observers)+1) :
        final[t] =normalize(np.multiply(alpha[t],beta[t]))
        #print("alpha ",alpha[t]," beta ", beta[t]," multip ", np.argmax(final[t]))
    return final

In [140]:
Obs=[0,0,1,0,0]
#print(forward_backward_impr1(U[0], A, O_true, pi))
print(forward_backward_impr1(Obs, A, O_true, pi))

[[0.6469432  0.3530568 ]
 [0.86734045 0.13265955]
 [0.82041955 0.17958045]
 [0.3074837  0.69251627]
 [0.8204191  0.17958093]
 [0.8673389  0.1326611 ]]
